## Dashboard for exploring glider section

Here we create a panel dashboard that allows us to explore the pressure-time section of the variables measured by the glider.

In [ ]:
import hvplot.xarray 
import numpy as np
import panel as pn
import xarray as xr
import holoviews as hv
from holoviews import opts
from holoviews import streams
import matplotlib.pyplot as plt
import param

In [ ]:
# Load Data 

data_dir = './data/'

ds_659_Tgrid = xr.open_dataset(data_dir + '659_Tgrid.nc')
ds_660_Tgrid = xr.open_dataset(data_dir + '660_Tgrid.nc')

ds_659_Tgrid_anom = xr.open_dataset(data_dir + '659_Tgrid_anomaly.nc')
ds_660_Tgrid_anom = xr.open_dataset(data_dir + '660_Tgrid_anomaly.nc')

ds_659_Dgrid = xr.open_dataset(data_dir + '659_Dgrid.nc')
ds_660_Dgrid = xr.open_dataset(data_dir + '660_Dgrid.nc')

ds_659_Dgrid_anom = xr.open_dataset(data_dir + '659_Dgrid_anomaly.nc')
ds_660_Dgrid_anom = xr.open_dataset(data_dir + '660_Dgrid_anomaly.nc')

# These files were created before by interpolating from the glider's
# see-saw sampling to a uniform time-pressure grid or distance-pressure grid. 
# Code in data_preprocess_2

In [ ]:
glider_nums = ['sg659', 'sg660']
glider_vars = list(ds_659_Tgrid.keys()) # just need to do once bevause all data sets have same variables
#cmap_options = ['RdBu_r', 'Purples', 'YlGnBu', 'Greens', 'YlOrBr']
cmap_options = plt.colormaps()

In [ ]:
var_select_map = {
            'oxygen': {
                'bin_range': (175, 375), # remove these since we don't use these anymore 
                'cmap_sel': 'YlOrBr'
            },
            'Chl': {
                'bin_range': (0, 0.5),
                'cmap_sel': 'Greens'
            },
            'salinity': {
                'bin_range': (33.75, 35),
                'cmap_sel': 'YlGnBu'
            },
            'temperature': {
                'bin_range': (0,4),
                'cmap_sel': 'RdBu_r'
            },
            'potdens': {
                'bin_range': (1026.8, 1027.8),
                'cmap_sel': 'Purples'
            }
        }
# For future versions would be nice if some of these things are read in from the
# attributes. 

glider_map = {
            'sg659': {'Tgrid': ds_659_Tgrid, 'Dgrid': ds_659_Dgrid},
            'sg660': {'Tgrid': ds_660_Tgrid, 'Dgrid': ds_660_Dgrid},
        }

glider_map_anom = {
            'sg659': {'Tgrid': ds_659_Tgrid_anom, 'Dgrid': ds_659_Dgrid_anom},
            'sg660': {'Tgrid': ds_660_Tgrid_anom, 'Dgrid': ds_660_Dgrid_anom},
        }


In [ ]:
# working 
class GliderPlot(param.Parameterized):
    # Setup all the parameters for the widgets
    glider_num = param.Selector(glider_nums, default='sg659', 
                                label='Glider Num#', precedence=0)
    glider_grid = param.Selector(['Tgrid', 'Dgrid'], default='Tgrid', 
                                label='Grid Type', precedence=0)
    glider_var = param.Selector(glider_vars, default='temperature', 
                                label='Glider Variable', precedence=1)
    var_colormap = param.Selector(default='RdBu_r', objects=cmap_options, 
                                  label='Colormap', precedence=2)
    time_slider = param.Range(label='Days in 2019', 
                             bounds=(119, 209), 
                             default=(119, 135), precedence=3)
    distance_slider = param.Range(label='Along Track Distance',
                                 bounds=(0, 2e3), default=(0, 400), 
                                 precedence=-1) # start with a negative precedence, in accordance with default being Tgrid
    anomaly_boolean = param.Boolean(default=False, label='Anomaly', precedence=3)
    density_boolean = param.Boolean(default=True, label='Show Density Contours', precedence=4)
    density_range = param.Range(label='Density range', bounds=(1026.8, 1027.9), default=(1026.8, 1027.9),precedence=10)
    density_gradation = param.Integer(label='Density levels', default=11, bounds=(2, 21),precedence=10)
    
    rangex = streams.RangeX(x_range=(0,0))
    
#    rangex.x_range = (0,0)
    title =''
    
    ## The Methods
    # function to not have active toolbars by default
    def _set_tools(self, plot, element):
        plot.state.toolbar.active_drag = None
        plot.state.toolbar.active_inspect = None
    
    # function to update default colormap choices with changing variables
    @param.depends('glider_var', watch=True)
    def _update_colormap(self):
        self.var_colormap = var_select_map[self.glider_var]['cmap_sel']
    
    # The next couple of functions toggle the widgets visible or not. 
    @param.depends('density_boolean', watch=True)
    def _update_density_widgets(self):
        # helps to hide widgets when they are not being used. 
        if self.density_boolean:
            self.param.density_range.precedence=10
            self.param.density_gradation.precedence=10
        else:
            self.param.density_range.precedence=-1
            self.param.density_gradation.precedence=-1
            
    @param.depends('glider_grid', watch=True)
    def _update_grid_widgets(self):
        if self.glider_grid == 'Tgrid':
            self.param.time_slider.precedence=3
            self.param.distance_slider.precedence=-1
        else: 
            self.param.time_slider.precedence=-1
            self.param.distance_slider.precedence=3
            
    # function to make density contours
    @param.depends('density_range', 'density_gradation', 'glider_grid','glider_num')
    def density_contours(self):
        #print('in contour')
        contour = glider_map[self.glider_num][self.glider_grid]['potdens'].hvplot.contour(flip_yaxis=True, 
                                                                     levels=np.linspace(self.density_range[0],
                                                                                        self.density_range[1],
                                                                                        self.density_gradation)
                                                                       ).opts(tools=[])
        return contour
    
    # function to make the image for the glider section
    @param.depends('anomaly_boolean', 'glider_grid', 'glider_num', 'glider_var')
    def glider_image(self):
        # Change the data set if wanting to plot anomaly
        if self.anomaly_boolean:
            glid_ds = glider_map_anom
        else:
            glid_ds = glider_map

        # plot the image in Distance or Time
        if self.glider_grid=='Dgrid':
            image = hv.Image( (glid_ds[self.glider_num][self.glider_grid].distance, 
                           glid_ds[self.glider_num][self.glider_grid].pressure,
                           glid_ds[self.glider_num][self.glider_grid][self.glider_var]), 
                             ['Distance [km]', 'Pressure [dBar]'], self.glider_var)
            self.rangex.source = image
        else:
            image = hv.Image( (glid_ds[self.glider_num][self.glider_grid].time, 
                           glid_ds[self.glider_num][self.glider_grid].pressure,
                           glid_ds[self.glider_num][self.glider_grid][self.glider_var]), 
                             ['Time [days]', 'Pressure [dBar]'], self.glider_var)
            self.rangex.source = image
        
        # estimate the color range so that outliers don't create problems
        bin_range = np.nanpercentile(glid_ds[self.glider_num][self.glider_grid][self.glider_var], [.5,99.5])
        
        # set properties for image like colorbar etcs.
        image = image.opts(opts.Image(
                        colorbar=True,
                        cmap=self.var_colormap,
                        invert_yaxis=True,
                        clim=(bin_range[0], bin_range[1]),
                        width=800,
                        tools=['hover'], hooks=[self._set_tools]
                        ) )
        return image
    
    # function to update the title
#     @param.depends('rangex.params', watch=True)
#     def _update_title(self):
#         self.title = str(self.rangex.x_range)
        #print(self.rangex.x_range)
        
    # function to set the xlimits 
    #@param.depends('glider_grid', 'time_slider', 'distance_slider')
    #def adjusted_xlim_image(self):
     

        # add the adjoint histogram
        #image = image.hist()# bin_range = (bin_range[0], bin_range[1]) )
#         return image 
    
   # @param.depends('distance_slider', 'time_slider', watch=True)
    def viewable(self):
        # add the density contours or not. 
        #if self.density_boolean:
        #    return hv.DynamicMap(self.adjusted_xlim_image)*hv.DynamicMap(self.density_contours)
        #else:
        image = hv.DynamicMap(self.glider_image).hist()
        
        if self.glider_grid=='Dgrid':
            image.opts(opts.Image(xlim = self.distance_slider))
        else:
            image.opts(opts.Image(xlim = self.time_slider))

        return image*hv.DynamicMap(self.density_contours)
            

In [ ]:
test = GliderPlot()

In [ ]:
#pn.Column(test.param, test.viewable())
#pn.Row(test.param, test.viewable())
pn.Row(test.param, test.viewable)
# it makes a big difference if test.viewable is passed or test.viewable() is passed. 
# In the latter the dependencies are not automatically inferred. 
# In it's current form - using .viewable, even though we have explicitly defined 
# dependencies for many things in the class. A lot of the code gets called when widgets change. 

#https://examples.pyviz.org/datashader_dashboard/dashboard.html#datashader-dashboard-gallery-dashboard